# 複数のリトリーバーからの結果を統合する方法

[EnsembleRetriever](https://python.langchain.com/api_reference/langchain/retrievers/langchain.retrievers.ensemble.EnsembleRetriever.html) は、複数のリトリーバーからの結果を統合するための機能を提供します。  
このクラスは、複数の [BaseRetriever](https://python.langchain.com/api_reference/core/retrievers/langchain_core.retrievers.BaseRetriever.html)  オブジェクトをリストとして受け取り、それらの結果を [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf)アルゴリズム に基づいて再ランク付けします。

異なるアルゴリズムの強みを活用することで、`EnsembleRetriever` は単一のアルゴリズムよりも優れたパフォーマンスを実現できます。

最も一般的なパターンは、スパースリトリーバー（例: BM25）と密リトリーバー（例: 埋め込み類似性）の結果を組み合わせることです。  
この組み合わせは、それぞれの強みを補完し合うため、「ハイブリッド検索」として知られています。

- スパースリトリーバー
    - キーワードに基づいて関連性の高いドキュメントを見つけるのが得意。
    - 例: BM25。
- 密リトリーバー
    - 意味的な類似性に基づいて関連性の高いドキュメントを見つけるのが得意。
    - 例: 埋め込みモデルによる類似検索。

## 基本的な使用例

以下では、[BM25Retriever](https://python.langchain.com/api_reference/community/retrievers/langchain_community.retrievers.bm25.BM25Retriever.html) と [FAISS vector store](https://python.langchain.com/api_reference/community/vectorstores/langchain_community.vectorstores.faiss.FAISS.html)に基づくリトリーバーを組み合わせた例を示します。

In [9]:
%pip install --upgrade --quiet  rank_bm25 > /dev/null

In [3]:
from langchain.retrievers import EnsembleRetriever  # 複数のリトリーバーを統合するためのクラス
from langchain_community.retrievers import BM25Retriever  # スパースリトリーバー（BM25）を提供
from langchain_community.vectorstores import FAISS  # ベクトルストア（FAISS）を提供
from langchain_openai import OpenAIEmbeddings  # OpenAIの埋め込みモデルを利用

# --- データセット 1: BM25リトリーバー用のドキュメントリスト ---
doc_list_1 = [
    "I like apples",  # ドキュメント 1
    "I like oranges",  # ドキュメント 2
    "Apples and oranges are fruits",  # ドキュメント 3
]

# BM25リトリーバーの初期化
bm25_retriever = BM25Retriever.from_texts(
    doc_list_1, metadatas=[{"source": 1}] * len(doc_list_1)  # 各ドキュメントにメタデータを付与
)
bm25_retriever.k = 2  # 上位2件の結果を返す設定

# --- データセット 2: FAISSリトリーバー用のドキュメントリスト ---
doc_list_2 = [
    "You like apples",  # ドキュメント 1
    "You like oranges",  # ドキュメント 2
]

# FAISSリトリーバーの初期化
embedding = OpenAIEmbeddings()  # 埋め込みモデルを設定
faiss_vectorstore = FAISS.from_texts(
    doc_list_2, embedding, metadatas=[{"source": 2}] * len(doc_list_2)  # 各ドキュメントにメタデータを付与
)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})  # 上位2件の結果を返す設定

# --- EnsembleRetrieverの初期化 ---
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],  # BM25リトリーバーとFAISSリトリーバーを組み合わせる
    weights=[0.5, 0.5]  # 両リトリーバーの結果に同じ重み付け
)


In [4]:
docs = ensemble_retriever.invoke("apples")
docs

[Document(page_content='I like apples', metadata={'source': 1}),
 Document(page_content='You like apples', metadata={'source': 2}),
 Document(page_content='Apples and oranges are fruits', metadata={'source': 1}),
 Document(page_content='You like oranges', metadata={'source': 2})]

## 
実行時の設定
　
個々のリトリーバーを実行時に設定することも可能です。設定可能なフィールドを使用して行います。  
以下は、FAISSリトリーバーの「top-k」パラメータを特定の値に更新する例です：

In [5]:
from langchain_core.runnables import ConfigurableField

faiss_retriever = faiss_vectorstore.as_retriever(
    search_kwargs={"k": 2}
).configurable_fields(
    search_kwargs=ConfigurableField(
        id="search_kwargs_faiss",
        name="Search Kwargs",
        description="The search kwargs to use",
    )
)

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

In [6]:
config = {"configurable": {"search_kwargs_faiss": {"k": 1}}}
docs = ensemble_retriever.invoke("apples", config=config)
docs

[Document(page_content='I like apples', metadata={'source': 1}),
 Document(page_content='You like apples', metadata={'source': 2}),
 Document(page_content='Apples and oranges are fruits', metadata={'source': 1})]

これにより、FAISSリトリーバーからのソースが1つだけ返されることに注意してください。これは、実行時に関連する設定を渡しているためです。